In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from umap import UMAP
from Bio import SeqIO
import bioframe
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import h5py
os.environ['CUDA_VISIBLE_DEVICES']='1'
from transformers import AutoModel,AutoTokenizer,AutoModelForMaskedLM
from tqdm import tqdm
import torch
import gpn.mlm
import sys
sys.path.append('../')
import utils
#gonzalobenegas/gpn-arabidopsis
#model = AutoModel.from_pretrained("/home/amber/multitask_RNA/replications/GPN/GPN_finetune/checkpoint-2000000").to('cuda')
#tokenizer = AutoTokenizer.from_pretrained("/home/amber/multitask_RNA/replications/GPN/GPN_finetune/checkpoint-2000000")

# model = AutoModel.from_pretrained("gonzalobenegas/gpn-arabidopsis").to('cuda')
# tokenizer = AutoTokenizer.from_pretrained("gonzalobenegas/gpn-arabidopsis")

model = AutoModel.from_pretrained("/home/amber/multitask_RNA/replications/GPN/GPN_archive/checkpoint-80000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("/home/amber/multitask_RNA/replications/GPN/GPN_archive/checkpoint-80000")

model.eval();
file = h5py.File('../../data/rna_stable/insert_dataset.h5','r')

Some weights of the model checkpoint at /home/amber/multitask_RNA/replications/GPN/GPN_archive/checkpoint-80000 were not used when initializing ConvNetModel: ['cls.decoder.3.weight', 'cls.decoder.0.weight', 'cls.decoder.2.bias', 'cls.decoder.3.bias', 'cls.decoder.2.weight', 'cls.decoder.0.bias']
- This IS expected if you are initializing ConvNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ConvNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
gpn_output = h5py.File('../../data/rna_stable/gpn_human_embed.h5','w')
batch_size = 32
for dataset in ['test','train','valid']:
    key = 'X_'+dataset
    onehot = file[key]
    string_seq = utils.onehot_to_seq(onehot)

    token_seq = tokenizer.batch_encode_plus(string_seq, max_length=512,padding = 'max_length')
    output_cache = []
    for seq_i in tqdm(range(0,len(token_seq['input_ids']),batch_size)):
        seq_batch = torch.tensor(token_seq['input_ids'][seq_i:seq_i+batch_size]).to('cuda')
        output_seq = model(seq_batch).last_hidden_state.cpu().detach().numpy()
        output_cache.extend(output_seq[:,:173,:])
    gpn_output.create_dataset(name=key,data = np.array(output_cache))
    gpn_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:05<00:00,  6.09it/s]


In [7]:
gpn_output.close()

In [11]:
np.array(output_cache).shape

(1149, 173, 512)